# GCC's ssc-cdi : tomography pipeline for Cateretê v0.1
 
- #### Questions? E-mail gcc@lnls.br
- #### Documentation about the ssc-cdi package: https://gcc.lnls.br/wiki/
- #### To use a test sample:
    - dic["sinogram_path"] = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/phantom_complex/phantom_complex.hdf5"

## Imports

In [1]:
%matplotlib widget
import numpy as np, matplotlib.pyplot as plt
from sscCdi import read_data, define_paths, tomo_sort, tomo_crop, tomo_unwrap, tomo_equalize, preview_angle_projection, tomo_alignment, tomo_recon, tomo_equalize3D, deploy_visualizer

## Load dictionary

In [2]:
dic = {} # or load from template file

## Load data

In [4]:
dic["recon_method"]  = "ptycho" # ptycho or pwcdi
dic["contrast_type"] = "phase" # phase or magnitude
dic["sinogram_path"] = "/ibira/lnls/labs/tepui/home/yuri.tonin/00000000/proc/recons/glass24/2023-05-12-15h19m_glass24.hdf5"

dic = define_paths(dic)
obj, angles = read_data(dic)
print(f"Object shape = {obj.shape} \t Number of angles: {angles.shape}")

Object shape = (60, 3481, 3481) 	 Number of angles: (60, 2)


In [4]:
deploy_visualizer(obj,title='Original sinogram',cmap='gray')

In [5]:
crop = obj[:,1600:-1400,1400:-1500]
deploy_visualizer(crop,title='Original sinogram',cmap='gray')

## Sort data

In [ ]:
tomo_sort(dic,obj, angles)

In [8]:
deploy_visualizer(np.load(dic["ordered_object_filepath"]),title="Ordered sinogram",cmap='gray')

## Crop

In [9]:
dic["top_crop"]    = 1600
dic["bottom_crop"] = 1400
dic["left_crop"]   = 1400
dic["right_crop"]  = 1500
tomo_crop(dic)

Cropped sinogram shape: (60, 481, 581)
Time elapsed: 1.14 s


In [10]:
deploy_visualizer(np.load(dic["cropped_sinogram_filepath"]),title="Cropped sinogram",cmap='gray')

## Unwrap

In [13]:
dic["bad_frames_before_unwrap"] = []
tomo_unwrap(dic)

Using 32 parallel processes


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 146.80it/s]


Time elapsed: 4.32 s


In [14]:
deploy_visualizer(np.load(dic["unwrapped_sinogram_filepath"]),title="Unwrapped sinogram",cmap='gray')

## 2D Equalization

In [15]:
dic["bad_frames_before_equalization"] = []
dic["equalize_gradient"] = 0
dic["equalize_global_offset"] = False
dic["equalize_invert"] = True
dic["equalize_local_offset"] = [ 400, 450,500, 550 ]
dic["equalize_outliers"] = 0

tomo_equalize(dic)

Sinogram shape to unwrap:  (60, 481, 581)
Using 32 parallel processes


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 2083.07it/s]


Min 	 Mean-3*sigma 	 Mean 	 Mean+3*sigma 	 Max 
Old  0.0 -4.760403752326965 1.3935772 7.5475581884384155 6.892859
New  0.0 -4.844692945480347 1.202848 7.250388860702515 7.22563
Time elapsed: 0.64 s


In [16]:
deploy_visualizer(np.load(dic["equalized_sinogram_filepath"]),title="Equalized sinogram",cmap='gray')

In [31]:
deploy_visualizer(np.load(dic["equalized_sinogram_filepath"]),title="Equalized sinogram",cmap='gray',axis=1)

## Alignment

In [36]:
from sscCdi import angle_grid_organize
angles  = np.load(dic["ordered_angles_filepath"])
object = np.load(dic["wiggle_sinogram_selection"]) 

object, _, _, projected_angles = angle_grid_organize(object, angles,percentage=dic["step_percentage"])

In [38]:
projected_angles

array([  0.,  30.,  60.,  90., 120., 150.])

In [26]:
dic["wiggle_sinogram_selection"] = dic["equalized_sinogram_filepath"]
dic["step_percentage"] = 1 # need to project irregular angle steps to a regular grid?
preview_angle_projection(dic)

Simulating projection of angles to regular grid...
Before+after frames added: 59
Intermediate null frames : 188
                        + -----
Total null frames        : 247
Frames being used        : 51  of  60
                        + -----
Projected Angles         : 298


In [27]:
dic["CPUs"] = 32
dic["bad_frames_before_wiggle"] = [] 
dic["wiggle_reference_frame"] = 0
dic = tomo_alignment(dic)

ssc-radon: creating 6x481x581 shared arrays (tomogram)
ssc-radon: creating 6x481x581 shared arrays (tomogram)
	Finished vertical wiggle. Starting horizontal wiggle...
ssc-radon: creating 6x481x581 shared arrays (tomogram)
Time elapsed: 0.90 s


In [28]:
deploy_visualizer(np.load(dic["wiggle_sinogram_filepath"]),title="Aligned sinogram",cmap='gray',axis=0)

## Tomography

In [ ]:
dic["GPUs"] = [0]
dic["tomo_algorithm"] = "EEM"
dic["tomo_iterations"] = 40
dic["tomo_regularization"] = False
dic["tomo_regularization_param"] = 0.001

dic = tomo_recon(dic)

In [ ]:
deploy_visualizer(np.load(dic["reconstruction_filepath"]),title="Recon",cmap='gray',axis=0)

## 3D Equalization

In [ ]:
dic["tomo_remove_outliers"] = 0
dic["tomo_threshold"] = 1000.0
dic["tomo_local_offset"] = []
dic["tomo_mask"] = []

dic = tomo_equalize3D(dic)

In [ ]:
deploy_visualizer(np.load(dic["eq_reconstruction_filepath"]),title="Equalized Recon",cmap='gray',axis=0)